# Boolean Query

In this notebook, we will implement a Boolean information retrieval model by Elasticsearch.  
We have these steps: <br>

**1- Connect to the Elasticsearch Cluster and Create an Index <br>
2- Indexing Documents <br>
3- Boolean Retrieval (Fill where ever it says #TODO in this part)** <br>

Let me know if you have any problems with this notebook or implementation. <br>
**Telegram**: [@Mohammad_Ardestani4](https://t.me/Mohammad_Ardestani4) <br>
**Email**: mjavad.ardestani00@gmial.com

In [1]:
!pip install elasticsearch

In [2]:
from elasticsearch import Elasticsearch, helpers
import os, json, time
from datetime import datetime
from tqdm import tqdm 

## Ignore The Security Warnings
Here we ignore the security warnings. It's suggested that you don't run below cell until you ensure that all warnings are about security. 


In [3]:
import warnings
warnings.filterwarnings('ignore')

## Load Processed Data

In [4]:
# load from .csv file
import pandas as pd
df = pd.read_csv('preprocessed_data.csv',index_col=0)
df = df.to_dict(orient='records')
data = {str(k): v for k, v in enumerate(df)}

In [ ]:
# load from .json file
import json
dataset_path = 'IR_data_news_12k.json'
with open(dataset_path) as f:
    data = json.load(f)
print(data['0'].keys())

In [5]:
data['0'].keys()

dict_keys(['title', 'content', 'tags', 'date', 'url', 'category'])

In [6]:
data['0'].items()

dict_items([('title', 'اعلام زمان قرعه کشی جام باشگاه های فوتسال آسیا'), ('content', 'گزار خبرگزار فارس کنفدراسیون فوتبال آسیا AFC نامه رسم فدراسیون فوتبال ایر باشگاه گیت پسند ز قرعه کش جا باشگاه فوتسال آسیا رسما اعلا اساس ۲۵ فروردین ماه ۱۴۰۱ مراس قرعه کش جا باشگاه فوتسال آسیا مالز برگزار باشگاه گیت پسند بعنو قهر فوتسال ایر سال ۱۴۰۰ مسابق گیت پسند تجربه ۳ دوره حضور جا باشگاه فوتسال آسیا دوره فینال مسابق عنو قهرمان مقا دوم بدس آورده_اس انت پیا'), ('tags', "['اعلام زمان', 'قرعه\\u200cکشی', 'قرعه\\u200cکشی جام', 'قرعه\\u200cکشی جام باشگاه\\u200cهای فوتسال', 'ای اف سی', 'گیتی پسند']"), ('date', '3/15/2022 5:59:27 PM'), ('url', 'https://www.farsnews.ir/news/14001224001005/اعلام-زمان-قرعه-کشی-جام-باشگاه-های-فوتسال-آسیا'), ('category', 'sports')])

## Connect to the Elasticsearch Cluster and Create an Index
After starting your Elasticsearch on your pc (localhost:9200 is the default), we have to connect to it via the following piece of code.


In [7]:
index_name = 'fars-news-retrieval'
es = Elasticsearch("http://localhost:9200")
es.indices.create(index = index_name)

BadRequestError: BadRequestError(400, 'resource_already_exists_exception', 'index [fars-news-retrieval/L3EhkTDXTqOZ7eMeEdHFjA] already exists')

### Check the Cluster Status

In [8]:
dict(es.info())

{'name': 'Ali',
 'cluster_name': 'elasticsearch',
 'cluster_uuid': 'XpCit1UoTwWArWjcHemYuQ',
 'version': {'number': '8.3.0',
  'build_type': 'deb',
  'build_hash': '5b8b981647acdf1ba1d88751646b49d1b461b4cc',
  'build_date': '2022-06-23T22:48:49.607492124Z',
  'build_snapshot': False,
  'lucene_version': '9.2.0',
  'minimum_wire_compatibility_version': '7.17.0',
  'minimum_index_compatibility_version': '7.0.0'},
 'tagline': 'You Know, for Search'}

## Indexing Documents 

we can add our documents to created index in two ways:<br>
1- one by one in for loop. <br>
2- use [Bulk API](https://www.elastic.co/guide/en/elasticsearch/reference/current/docs-bulk.html#:~:text=Bulk%20APIedit,can%20greatly%20increase%20indexing%20speed.), which performs multiple indexing or delete operations in a single API call. This reduces overhead and can significantly increase indexing speed.

###  for loop by es.index 
**You dont have to run this cell**. 

In [9]:
for i in tqdm(range(len(data))):
    es.index(index = index_name, id=i, document=data[str(i)])
    i = i + 1

100%|████████████████████████████████████| 12202/12202 [01:03<00:00, 191.90it/s]


###  Bulk API

In [32]:
from elasticsearch.helpers import bulk

def bulk_sync():
    actions = [
        {
            '_index': index_name,
            '_id':doc_id,
            '_source': doc
        } for doc_id,doc in data.items()
    ]
    bulk(es, actions)


In [33]:
start = time.time()
bulk_sync()
end = time.time()
print("Indexing all documents took about {:.2f} seconds".format(end - start))

Indexing all documents took about 2.46 seconds


### check index

In [34]:
es.count(index = index_name)

ObjectApiResponse({'count': 12202, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}})

## Boolean Retrieval 

### Bool Query Structure

It is built using one or more boolean clauses, each clause with a typed occurrence. The occurrence types are:

**must**: The clause (query) must appear in matching documents and will contribute to the score.
<br><br>
**filter**: The clause (query) must appear in matching documents. However unlike must the score of the query will be ignored. Filter clauses are executed in filter context, meaning that scoring is ignored and clauses are considered for caching.
<br><br>
**should**: The clause (query) should appear in the matching document.
<br><br>
**must_not**: The clause (query) must not appear in the matching documents. Clauses are executed in filter context meaning that scoring is ignored and clauses are considered for caching. Because scoring is ignored, a score of 0 for all documents is returned.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-bool-query.html). 

###  <span style="color:red"> TODO </span>
You should read about [match query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query.html#match-top-level-params) and [match phrase query](https://www.elastic.co/guide/en/elasticsearch/reference/current/query-dsl-match-query-phrase.html) then complete the below cell.

In [35]:
from hazm import Normalizer, word_tokenize, stopwords_list, Stemmer
def preprocess_query(query):
    """
        Preprocesses a query.
        normalizes and tokenizes the query.
        removes stopwords, punctuation, urls ,and stemming.
    """
    query = word_tokenize(query)
    query = [w for w in query if w not in stopwords_list()]
    query = [Stemmer().stem(w) for w in query]
    return ' '.join(query)

In [36]:
query1= {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match" : {"content" : preprocess_query("آمریکا")}
                  
              }, 
              
              { 
                  #TODO: add a match pharse query strucutre ==> use for words in <"">
                  "match_phrase" : {"content" : preprocess_query("تحریم هسته‌ای")}
              },
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match" : {"content" : preprocess_query("ایران")}
              }
          ],
        },
    }

In [37]:
query2= {
        "bool": {
          "should": [
              { 
                  #TODO: add a match query structure ==> use for normal words
                  "match" : {"content" : preprocess_query("اورشلیم")}
                  
              }
   
          ],
          "must_not": [
              {
                  #TODO: add a match or match pharse query structure ==> use for words after <!>
                  "match" : {"content" : preprocess_query("صهیونیست")}
              }
          ],
        },
    }

### Search query

The _source field contains the original JSON document body that was passed at index time. The _source field itself is not indexed (and thus is not searchable), but it is stored so that it can be returned when executing fetch requests, like get or search.

For further information, you can read this [Document](https://www.elastic.co/guide/en/elasticsearch/reference/current/search-search.html#search-api-response-body).

In [38]:
res1 = es.search(index = index_name, query=query1, _source= ["url"])
res1 = dict(res1)

In [39]:
res2 = es.search(index = index_name, query=query2, _source= ["url"])
res2 = dict(res2)

### Results

In [40]:
print("{} results in {} s: ".format(res1['hits']['total']['value'] ,res1['took']/1000))
for doc in res1['hits']['hits']:
    print(doc['_source']['url'])

221 results in 0.006 s: 
https://www.farsnews.ir/news/14001214000789/نماینده-کلیمیان-در-مجلس-دهم--کارتل‌ها-بر-تصمیمات-هیأت-حاکمه-آمریکا
https://www.farsnews.ir/news/14001213000089/آمریکا-با-بحران‌آفرینی-به-حیات-خود-ادامه-می-دهد
https://www.farsnews.ir/news/14001211000898/سود-مافیای-اسلحه‌سازی-آمریکا-در-ناامن-بودن-جهان-است
https://www.farsnews.ir/news/14001211000321/آمریکا-دولت‌های-متحد-خود-را-در-زمان-اضطرار-تنها-می‌گذارد
https://www.farsnews.ir/news/14001217000665/خباثت‌های-آمریکا-در-برجام-روسیه-و-چین-را-هم-به-این-کشور-بدبین-کرد
https://www.farsnews.ir/news/14001214000825/آمریکا-رژیمی-مافیایی-است-و-مردم-در-تصمیمات-حاکمان-آن-جایگاهی-ندارند
https://www.farsnews.ir/news/14001212000725/آمریکا-برای-فروش-تسلیحات-خود-به-ایجادناامنی-و-بحران‌آفرینی-نیاز-دارد
https://www.farsnews.ir/news/14001211000220/نماینده-کلیمیان-در-مجلس-منافع-رژیم-مافیایی-آمریکا-در-ایجاد-ناامنی-است
https://www.farsnews.ir/news/14001213000328/کارتل‌های-اقتصادی-رؤسای-جمهور-آمریکا-را-تعیین-می‌کنند
https://www.farsnews.ir/news

In [41]:
print("{} results in {} s: ".format(res2['hits']['total']['value'] ,res2['took']/1000))
for doc in res2['hits']['hits']:
    print(doc['_source']['url'])

0 results in 0.001 s: 
